# 2. Features
definicion de features a utilizar y luego el proceso de extraccion de los mismos

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("cb_mct.csv")
df.head()

,danho,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Cod.Curso,Convocatoria,Anho,Semestre,Aprobado,...,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,Tercer.Par,Asis,Cuarto.Par,4P_60,id_anony
0,2012,2,9001,CALCULO 1,MCT-PLS09,1,1,2012,2,S,...,24,3,0,0,2.0,0.0,1.0,NaN,NaN,es_1366
1,2012,2,9001,CALCULO 1,MCT-PLS09,1,1,2012,2,N,...,6,1,0,0,0.0,0.0,1.0,NaN,NaN,es_1510
2,2012,2,9001,CALCULO 1,MCT-PLS09,1,1,2012,2,N,...,6,3,0,0,0.0,0.0,1.0,NaN,NaN,es_1514
3,2012,2,9001,CALCULO 1,MCT-PLS09,1,1,2012,2,N,...,18,4,0,0,1.0,0.0,1.0,NaN,NaN,es_1431
4,2012,2,9001,CALCULO 1,MCT-PLS09,1,1,2012,2,S,...,22,3,39,0,2.0,0.0,1.0,NaN,NaN,es_1539


In [3]:
mct_cb = df.copy(deep=True)

In [4]:
mct_cb = mct_cb.drop_duplicates()

In [5]:
cant=np.unique(mct_cb['id_anony']) #alumnos de cb
print(len(cant)) #cantidad

632


In [6]:
sem_1 = mct_cb[mct_cb['Cod.Curso']==1]
materias = np.unique(sem_1['Asignatura']) # Asignaturas de 1er semestre
materias

array(['ALGEBRA LINEAL 1', 'CALCULO 1', 'DIBUJO TECNICO', 'FISICA 1',
       'GEOMETRIA ANALITICA', 'GEOMETRIA DESCRIPTIVA'], dtype=object)

In [10]:
materiaspaso=[] #Cantidad de materias que ya aprobo
pasoCB=[] # 1 si ya aprobo CB, 0 si aun no
tardocb=[] # Cantidad de años que tardo o esta tardando 
danhoini=[] # Año que curso
id_alumno=[] #Alumnos
codcarsec=[] # Codigo de Carrera
anhofin=[] #Año que finalizo
recursadas=[] #Cantidad de veces que recurso
recursadas_s1=[] # recursadas de 1er semestre
# mate_s1: promedio por materia
mate_s1={}
for mat in materias:
    mate_s1[mat]=[]
mate_s1
promedio_s1=[] #nota promedio del 1er semestre

In [11]:
def prom_mat(sem_1, mat): # retorna promedio por materia
    '''param: sem_1 = df de cada alumno
              mat = materia '''
    aux_1 = sem_1[sem_1['Asignatura']==mat]
    aux_2 = aux_1[aux_1['Nota.Final']!=0.0]
    prom = 0
    if len(aux_2)!=0.0:
        prom = sum(aux_2['Nota.Final'])/len(aux_2)
    return prom

def re_s1(sem_1,alum): # recursadas en 1er semestre
    aux_3 = sem_1[sem_1['id_anony']==alum]
    aux_4 = aux_3[aux_3['Aprobado']=='N']
    re = len(aux_4)
    return re

In [12]:
for alum in cant:
    paso = 0
    recurso = 0
    r_s1= 0
    pasCB=0
    aux= mct_cb[mct_cb['id_anony']==alum]
    sem_1 = aux[aux['Cod.Curso']==1]
    for mat in materias:
        mate_s1[mat].append(prom_mat(sem_1, mat))
    r_s1 = re_s1(sem_1,alum)
    sem_1 = sem_1[sem_1['Nota.Final']!=0.0]
    if len(sem_1)!=0.0:
        prom_1 = sum(sem_1['Nota.Final'])/len(sem_1)
    if len(sem_1)==0.0:
        prom_1 = 0
    plan= np.unique(aux['Cod.Car.Sec'])
    aux2= aux[aux['Aprobado']=='S']
    aux2=aux2.drop_duplicates(subset=['Asignatura'])# elimine las filas repetidas con misma Asignatura
    paso= len(aux2['Aprobado'])
    aux3=aux[aux['Aprobado']=='N']
    aux3=aux3.drop_duplicates(subset=['Asignatura','Anho','ciclo','danho'])#elimine filas con mismos datos
    recurso= len(aux3['Aprobado'])
    tardo = aux['Anho'].max() - aux['danho'].min() #es lo que tardo o esta tardando en pasar
    if plan[0] == 'MEC-PLS13' and paso>20:
        pasCB=1
    if plan[0]!='MEC-PLS13' and paso>19:
        pasCB=1
    materiaspaso.append(paso)
    tardocb.append(tardo)
    danhoini.append(aux['danho'].min())
    id_alumno.append(alum)
    codcarsec.append(plan[0])
    anhofin.append(aux['Anho'].max() )
    recursadas.append(recurso)
    pasoCB.append(pasCB)
    recursadas_s1.append(r_s1)
    promedio_s1.append(prom_1)

In [13]:
materias

array(['ALGEBRA LINEAL 1', 'CALCULO 1', 'DIBUJO TECNICO', 'FISICA 1',
       'GEOMETRIA ANALITICA', 'GEOMETRIA DESCRIPTIVA'], dtype=object)

In [14]:
new= pd.DataFrame({'id_anony':id_alumno,'Cod.Car.Sec':codcarsec,'Anho_in':danhoini,
    'Anho_out':anhofin,'tardoCB':tardocb,'PasoCB':pasoCB,'recurso':recursadas,'recursadas_s1':recursadas_s1,'Cant.Mat.Aprob':materiaspaso,
    'prom_al1':mate_s1[materias[0]], 'prom_c1':mate_s1[materias[1]], 'prom_dt':mate_s1[materias[2]],
    'prom_f1':mate_s1[materias[3]], 'prom_ga':mate_s1[materias[4]],'prom_gd':mate_s1[materias[5]],
    'prom_1er_S':promedio_s1 })

In [23]:
mct_paso_cb = new[new['PasoCB']==1] # alumnos que culminaron cb
mct_no_paso_cb = new[new['PasoCB']==1] # alumnos que todavia no culminan cb
mct_paso_cb

,id_anony,Cod.Car.Sec,Anho_in,Anho_out,tardoCB,PasoCB,recurso,recursadas_s1,Cant.Mat.Aprob,prom_al1,prom_c1,prom_dt,prom_f1,prom_ga,prom_gd,prom_1er_S
102,es_1756,MCT-PLS09,2012,2015,3,1,7,0,20,2.0,3.0,0.0,3.0,5.0,0.0,3.250000
107,es_1778,MCT-PLS09,2012,2016,4,1,8,0,20,1.5,3.0,0.0,2.0,4.0,0.0,2.625000
112,es_1815,MCT-PLS13,2013,2018,5,1,12,2,21,5.0,4.0,3.0,4.0,3.0,3.0,3.666667
116,es_1830,MCT-PLS13,2013,2015,2,1,1,0,20,4.0,2.0,3.0,4.0,5.0,5.0,3.833333
117,es_1841,MCT-PLS13,2013,2017,4,1,7,0,20,1.5,4.0,3.0,4.0,4.0,5.0,3.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,es_4037,MCT-PLS13,2018,2020,2,1,0,0,21,5.0,5.0,1.5,5.0,4.0,1.5,3.666667
499,es_4053,MCT-PLS13,2018,2020,2,1,0,0,21,4.0,5.0,3.0,5.0,4.0,2.0,3.833333
501,es_4066,MCT-PLS13,2018,2021,3,1,3,0,20,5.0,5.0,2.0,5.0,5.0,2.0,4.000000
508,es_4097,MCT-PLS13,2018,2021,3,1,2,0,20,3.0,4.0,3.0,3.0,3.0,3.0,3.166667


In [25]:
# hasta aca extraccion de features
mct_paso_cb.to_csv('mct_paso_cb.csv', index=False)
mct_no_paso_cb.to_csv('mct_no_paso_cb.csv', index=False)